In [13]:
import pandas as pd
import os
import joblib

In [16]:
feat_path = '/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/merged_features/features_sparse'
analysis_path = '/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/features_by_analysis'
cohort_path = '/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort'

In [3]:
def read_file(filename, columns=None, **kwargs):
    print(filename)
    load_extension = os.path.splitext(filename)[-1]
    if load_extension == ".parquet":
        return pd.read_parquet(filename, columns=columns,**kwargs)
    elif load_extension == ".csv":
        return pd.read_csv(filename, usecols=columns, **kwargs)

In [9]:
feats = read_file(
		os.path.join(
			feat_path,
			"features_row_id_map.parquet"
		),
		engine='pyarrow'
	)

/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/merged_features/features_sparse/features_row_id_map.parquet


In [18]:
age_group = read_file(
		os.path.join(
			analysis_path,
			"age_group/static/features_285.parquet"
		),
		engine='pyarrow'
	)

/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/features_by_analysis/age_group/static/features_285.parquet


In [19]:
age_group.head()

,prediction_id,person_id,index_date,concept_id,analysis_id,time_bin,feature_id,concept_count
0,-2170876333730134375,30630459,2019-07-25 16:15:00,age_group_18,age_group,static,age_group_18_age_group,1
1,-930060984611237818,30230397,2021-09-14 04:05:00,age_group_18,age_group,static,age_group_18_age_group,1
2,-5188106954279777439,42133663,2019-09-27 04:22:00,age_group_18,age_group,static,age_group_18_age_group,1
3,-5943670930084792567,31046784,2019-08-16 10:30:00,age_group_18,age_group,static,age_group_18_age_group,1
4,-337833218730288408,31044908,2021-03-23 13:34:00,age_group_18,age_group,static,age_group_18_age_group,1


In [14]:
joblib.load(os.path.join(feat_path,"features.gz"))

<292392x44905 sparse matrix of type '<class 'numpy.int64'>'
	with 24759472 stored elements in Compressed Sparse Row format>